## Classification
- **Culturally Exclusive**: 'caponata' -> Italy
- **Cultural Representative**: 'pizza' -> Italy
- **Cultural Agnostic**: 'bread'

#### Exploiting the graph knowledge-based wikidata structure
- **Entity**: the item (e.g., "pizza")
- **P495**: country of origin
- **P2596**: culture
- **P172**: ethnic group
- **P37**: official language
- **P407**: langauge of work or name
- **P135**: Movement(Art, Literature, philosophy)
- **P136**: Genre
- **P921**: Main subject
- **P547**: Memorialized by
- **P784**: Significant event
- **P840**: Narrative location
- **P17**: Country
- **P1843**: Taxon common name
- **P1001**: Applies to jurisdiction
- **P144**: Based on
- **P361**: Part of
- **P1705**: Native label
- **P2012**: cuisine
- **P2541**: Operating area
- **P1535**: Used by
- **P366**: Use
- **P1142**: Political ideology
- **P140**: Religion
- **P102**: Member of political party
- **P1344**: Participant in
- **P183**: Endemic to
- **P2341**: Indigenous to
- **P1532**: Country for sport
- **P279**: Subclass of


In [1]:
# import requests
# import json
# from wikidata.client import Client
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import time
from datasets import load_dataset, load_from_disk


In [2]:
properties = ['originLabel', 'cultureLabel', 'ethnic_groupLabel', 'off_languageLabel',
                'nameLabel', 'movementLabel', 'genreLabel', 'main_subjectLabel',
                'memorializedLabel', 'sign_eventLabel', 'narrative_locLabel', 'countryLabel',
                'taxonLabel', 'jurisdictionLabel', 'based_onLabel', 'part_ofLabel',
                'native_labelLabel', 'cuisineLabel', 'areaLabel', 'used_byLabel',
                'useLabel', 'political_ideoLabel', 'religionLabel', 'political_partyLabel', 
                'participant_inLabel', 'endemicLabel', 'indigenousLabel', 
                'country_sportLabel', 'subclass_ofLabel'
              ]

In [3]:
def get_query_result(data, properties):
    query_result = {}
    keys = data['results']['bindings'][0].keys()
    for prop in properties:
        if prop in keys:
            query_result[prop] = data['results']['bindings'][0][prop]['value']
        else:
            query_result[prop] = None

    return query_result

In [4]:
sparql = SPARQLWrapper('https://query.wikidata.org/sparql', agent='GjWikidataBot/1.0')
sparql.setReturnFormat(JSON)

In [ ]:
# dataset = load_dataset('sapienzanlp/nlp2025_hw1_cultural_dataset')
# dataset.save_to_disk('./training_dataset')

In [5]:
dataset = load_from_disk('./datasets')

In what follows, we are going to extract new information from wikidata. We start from the items in the training dataset and we write a SPARQL query for extracting some useful information regarding them.

In [7]:
# First we extract the items' IDs
rows = len(dataset['train'])
IDs = []
for index in range(rows):
    item_id = dataset['train'][index]['item'].strip().split('/')[-1]
    IDs.append(item_id)

In [ ]:
gold_IDs = []
gold_df = pd.read_csv('gold.csv')
for url in gold_df['item']:
    item_id = url.strip().split('/')[-1]
    gold_IDs.append(item_id)

0       http://www.wikidata.org/entity/Q15786
1      http://www.wikidata.org/entity/Q268530
2      http://www.wikidata.org/entity/Q216153
3         http://www.wikidata.org/entity/Q593
4      http://www.wikidata.org/entity/Q192185
                        ...                  
295     http://www.wikidata.org/entity/Q36180
296    http://www.wikidata.org/entity/Q156316
297     http://www.wikidata.org/entity/Q56911
298       http://www.wikidata.org/entity/Q377
299    http://www.wikidata.org/entity/Q615394
Name: item, Length: 300, dtype: object


In [ ]:
def retrieve_info(items):
    i = 1                # to handle sleeping time
    dict_training = {}   # dictionary storing query results
    num_items = len(items)
    k = 0
    while k < num_items:
        item = items[k]
        sparql.setQuery(
            f'''
            SELECT ?originLabel ?cultureLabel ?ethnic_groupLabel ?off_languageLabel
                    ?nameLabel ?movementLabel ?genreLabel ?main_subjectLabel
                    ?memorializedLabel ?sign_eventLabel ?narrative_locLabel ?countryLabel
                    ?taxonLabel ?jurisdictionLabel ?based_onLabel ?part_ofLabel
                    ?native_labelLabel ?cuisineLabel ?areaLabel ?used_byLabel
                    ?useLabel ?political_ideoLabel ?religionLabel ?political_partyLabel 
                    ?participant_inLabel ?endemicLabel ?indigenousLabel 
                    ?country_sportLabel ?subclass_ofLabel
            WHERE {{
                OPTIONAL {{ wd:{item} wdt:P495 ?origin .}}  
                OPTIONAL {{ wd:{item} wdt:P2596 ?culture .}}
                OPTIONAL {{ wd:{item} wdt:P172 ?ethnic_group .}}
                OPTIONAL {{ wd:{item} wdt:P37 ?off_language .}}
                OPTIONAL {{ wd:{item} wdt:P407 ?name .}}
                OPTIONAL {{ wd:{item} wdt:P135 ?movement .}}
                OPTIONAL {{ wd:{item} wdt:P136 ?genre .}}
                OPTIONAL {{ wd:{item} wdt:P921 ?main_subject .}}
                OPTIONAL {{ wd:{item} wdt:P547 ?memorialized .}}
                OPTIONAL {{ wd:{item} wdt:P793 ?sign_event .}}
                OPTIONAL {{ wd:{item} wdt:P840 ?narrative_loc .}}
                OPTIONAL {{ wd:{item} wdt:P17 ?country .}}
                OPTIONAL {{ wd:{item} wdt:P1843 ?taxon .}}
                OPTIONAL {{ wd:{item} wdt:P1001 ?jurisdiction .}}
                OPTIONAL {{ wd:{item} wdt:P144 ?based_on .}}
                OPTIONAL {{ wd:{item} wdt:P361 ?part_of .}}
                OPTIONAL {{ wd:{item} wdt:P1705 ?native_label .}}
                OPTIONAL {{ wd:{item} wdt:P2012 ?cuisine .}}
                OPTIONAL {{ wd:{item} wdt:P2541 ?area .}}
                OPTIONAL {{ wd:{item} wdt:P1535 ?used_by .}}
                OPTIONAL {{ wd:{item} wdt:P366 ?use .}}
                OPTIONAL {{ wd:{item} wdt:P1142 ?political_ideo .}}
                OPTIONAL {{ wd:{item} wdt:P140 ?religion .}}
                OPTIONAL {{ wd:{item} wdt:P102 ?political_party .}}
                OPTIONAL {{ wd:{item} wdt:P1344 ?participant_in .}}
                OPTIONAL {{ wd:{item} wdt:P183 ?endemic .}}
                OPTIONAL {{ wd:{item} wdt:P2341 ?indigenous .}}
                OPTIONAL {{ wd:{item} wdt:P1532 ?country_sport .}}
                OPTIONAL {{ wd:{item} wdt:P279 ?subclass_of .}}        

                SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],mul,en". }}

            }}

            LIMIT 1
        '''
        )

        data = ''
        try:
            data = sparql.queryAndConvert()
            query_result = get_query_result(data, properties)
            dict_training[k] = query_result
            k += 1

        except Exception as e:
            time.sleep(10 * i)
            i += 1
            continue
            
    
    return dict_training
        
        
    

In [ ]:
dict_training = retrieve_info(IDs)

In [27]:
pd_table = pd.DataFrame.from_dict(dict_training, orient='index')

In [36]:
pd_table.to_csv('values.csv', index=True, na_rep='None')